In [1]:
# import libraries
import os 
from dotenv import load_dotenv
import requests
import json
import pandas as pd
from datetime import datetime
import pytz

In [2]:
# get API key from .env file
load_dotenv()

api_key = os.environ.get("API_KEY")

In [3]:
# coordinates of Saimai district and Pathumwan district
coor = {
    "saimai": {"lat": "13.896", "lon": "100.672"},
    "pathumwan": {"lat": "13.740", "lon": "100.535"}
}

# parts that unnecessary from API
parts = "minutely,hourly,daily,alerts"
# system of units
units = "metric"

In [4]:
# API url
weather_url = f"https://api.openweathermap.org/data/3.0/onecall?lat={coor['saimai']['lat']}&lon={coor['saimai']['lon']}&exclude={parts}&units={units}&appid={api_key}"
air_url = f"https://api.openweathermap.org/data/2.5/air_pollution?lat={coor['saimai']['lat']}&lon={coor['saimai']['lon']}&appid={api_key}"

In [5]:
# get a message payload from API
weather_response = requests.get(weather_url)
air_response = requests.get(air_url)

In [6]:
# print status code
print(weather_response)
print(air_response)

<Response [200]>
<Response [200]>


In [7]:
# get dictionary result
weather_result = weather_response.json()
air_result = air_response.json()

In [8]:
# print result
print(weather_result)
print(air_result)

{'lat': 13.896, 'lon': 100.672, 'timezone': 'Asia/Bangkok', 'timezone_offset': 25200, 'current': {'dt': 1683833776, 'sunrise': 1683845501, 'sunset': 1683891346, 'temp': 29.8, 'feels_like': 35.26, 'pressure': 1009, 'humidity': 73, 'dew_point': 24.44, 'uvi': 0, 'clouds': 20, 'visibility': 10000, 'wind_speed': 1.54, 'wind_deg': 150, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}]}}
{'coord': {'lon': 100.672, 'lat': 13.896}, 'list': [{'main': {'aqi': 5}, 'components': {'co': 3177.64, 'no': 75.1, 'no2': 41.81, 'o3': 0, 'so2': 38.15, 'pm2_5': 87.39, 'pm10': 135.53, 'nh3': 11.53}, 'dt': 1683833776}]}


In [9]:
# empty weather result dictionary
weather_dict = {}

# Thai timezone
thai_tz = pytz.timezone("Asia/Bangkok")

# time field list
time_list = ["dt", "sunrise", "sunset"]

# for loop in every key of raw data
for key in weather_result["current"].keys():
    # if key is in time_list
    if key in time_list:
        unix_time = weather_result["current"][key]
        # convert to datetime
        local_time = datetime.fromtimestamp(unix_time, thai_tz).strftime("%Y-%m-%d %H:%M:%S")
        # add to result dictionary
        weather_dict[key] = local_time
    # if key is not in time_list and weather, wind_gust, rain
    elif key not in time_list + ["weather", "wind_gust", "rain"]:
        # add this key to result dictionary
        weather_dict[key] = weather_result["current"][key]
    # if key is weather
    elif key == "weather":
        main = weather_result["current"][key][0]["main"]
        description = weather_result["current"][key][0]["description"]
        # add weather - main to result dictionary
        weather_dict["weather_main"] = main
        # add weather - description to result dictionary
        weather_dict["weather_description"] = description
    # if raw data contains wind_gust
    elif key == "wind_gust":
        wind_gust = weather_result["current"][key]
        # add wind_gust to result dictionary
        weather_dict["wind_gust"] = wind_gust
    # if raw data contains rain
    elif key == "rain":
        rain = weather_result["current"][key]["1h"]
        # add rain to result dictionary
        weather_dict["rain"] = rain

In [11]:
# empty air quality result dictionary
air_dict = {}

# add AQI score to result dictionary
air_dict["aqi"] = air_result["list"][0]["main"]["aqi"]

# for loop in every components
for component in air_result["list"][0]["components"]:
    # add component to result dictionary
    air_dict[component] = air_result["list"][0]["components"][component]

In [12]:
weather_dict

{'dt': '2023-05-12 02:36:16',
 'sunrise': '2023-05-12 05:51:41',
 'sunset': '2023-05-12 18:35:46',
 'temp': 29.8,
 'feels_like': 35.26,
 'pressure': 1009,
 'humidity': 73,
 'dew_point': 24.44,
 'uvi': 0,
 'clouds': 20,
 'visibility': 10000,
 'wind_speed': 1.54,
 'wind_deg': 150,
 'weather_main': 'Clouds',
 'weather_description': 'few clouds'}

In [13]:
air_dict

{'aqi': 5,
 'co': 3177.64,
 'no': 75.1,
 'no2': 41.81,
 'o3': 0,
 'so2': 38.15,
 'pm2_5': 87.39,
 'pm10': 135.53,
 'nh3': 11.53}

In [14]:
# weather dataframe
pd.DataFrame(weather_dict, index=[0])

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,weather_main,weather_description
0,2023-05-12 02:36:16,2023-05-12 05:51:41,2023-05-12 18:35:46,29.8,35.26,1009,73,24.44,0,20,10000,1.54,150,Clouds,few clouds


In [15]:
# air quality dataframe
pd.DataFrame(air_dict, index=[0])

,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,5,3177.64,75.1,41.81,0,38.15,87.39,135.53,11.53
